<a href="https://colab.research.google.com/github/victory1217/HHA504_E2E/blob/main/HHA504_E2E.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**HHA 504- E2E Final Assignment** 
##Victoria Rodriguez Silva 
## 12/10/2021 


## **First step - Set up and Deploy a Virtual Machine** 
#### 1. Go to https://azure.microsoft.com/en-us/. 
#### 2. Navigate to the Virtual Machines service (by using search or going to your Azure Portal). 
#### 3. Select Create + Virtual Machine. 
#### 4. Create a name, password, and select needed preferences for your machine.
#### 5. Review and Create the Virtual Machine.
#### **Note, before proceeding with the next steps, go to networking and add the MySQL inbound port rule (port 3306)!**






## **Second Step- Access the Remote Machine via your Terminal**
#### 1. Open your terminal, type ssh (username for your instance)@(ip address for your instance) - the IP Address for your machine can be found on the overview page.
#### 2. Then, enter in the password created for the instance when prompted.
#### 3. Once you enter the remote machine (name of virtual machine will appear in green letters), type "sudo apt-get update".
#### 4. After your new machine has been updated, type "sudo apt install mysql-server mysql-client" to install mysql on your remote machine. 
#### 5. Once mysql has been installed, type "sudo mysql" to enter into the mysql application via the terminal. 
#### 6. **You are now in mysql!** Now, type "show databases" to see what data you currently have. 
#### 7. You will then create a new mysql user to connect with, by typing "CREATE USER ‘USERNAME'@'%' IDENTIFIED BY ‘PASSWORD’; 
#### 8. To grant this new user privileges, you will type "GRANT ALL PRIVILEGES ON *.* TO 'USERNAME'@'%' WITH GRANT OPTION; - This username is solely for the mysql user and should not be the username created for the instance. 
#### 9. Test to see if the new user was created successfully by typing "mysql -u 'USERNAME' -p". Enter password when prompted. 
#### 10. Create a new database by typing "create database NAME;" and confirm the database was made by typing "show databases" - Note, that this is the name you are giving the database that will be located within mysql. 


## **Third Step - Connect to the SQL Instance using Python / Jupyter Notebook**
#### 1. Go to Spyder (Python 3.8) or any Jupyter Notebook of choice 
#### 2. Type the following code. 

In [ ]:
## Import needed packages 

from sqlalchemy import create_engine
import sqlalchemy
import pandas as pd 

##Connect to SQL instance 

MYSQL_HOSTNAME = 'IP ADDRESS OF INSTANCE' 
MYSQL_USER = 'USERNAME'
MYSQL_PASSWORD = 'PASSWORD'
MYSQL_DATABASE = 'DATABASE NAME '

connection_string = f'mysql+pymysql://{MYSQL_USER}:{MYSQL_PASSWORD}@{MYSQL_HOSTNAME}/{MYSQL_DATABASE}'
engine = create_engine(connection_string)

## Load CSV File 

csvfile = pd.read_csv('/Users/victoria_rodriguez/Downloads/H1N1_Flu_Vaccines.csv')
csvfile.to_sql('H1N1_Flu_Vaccines', con=engine, if_exists ='append') 
##Note - checked on MySQL Workbench to confirm that the table was loaded


## **Fourth Step- Fix Connection Error** 
#### 1. After running this code in Jupyter Notebook, go back to your terminal and type "sudo nano /etc/mysql/mysql.conf.d/mysqld.cnf" to fix configurations for your sql instance. 
#### 2. Once this has been completed, restart mysql within the virtual machine by typing "/etc/init.d/mysql restart" 
#### 3. Re-run the code in your Jupyter Notebook and the csv file will be found within the created database as a table. 


## **Fifth Step- Create a logical "cold" backup of SQL file** 
#### 1. Create a second instance (another virutal machine) using Microsoft Azure's Virtual Machine service - use the steps found within step 1!  
#### 2. After all of the same configurations and port rules have been set, log in to be sure that the instance is running - ssh USERNAME@IP ADDRESS (for new machine) 
#### 3. Go back to the host instance (first virtual machine created) and type "sudo mysqldump -- all databases> dump.sql".
#### 4. Within the first instance, type "ls" to confirm that the file has been saved. 
#### 5. Log in to the second instance (newly created virtual machine). To create a copy of the sql dump file, type "scp dump.sql USERNAME@2ND IP ADDRESS:/home/USERNAME 
#### **Note, for easy management, the same username and password were used for both of these virtual machines!** - you do not have to do the same. 
#### 6. Push enter, type in the created password, and type "ls" to confirm that the backup sql file was created in the second instance. 


## **Sixth Step - Create a Trigger**
#### 1. For the assigment, this is the desired trigger we were asked to create. 
#### 2. Verify the following code. 

In [ ]:
## Create a Trigger 

SELECT * FROM H1N1_Flu_Vaccines LIMIT 15;
DELIMITER $$
CREATE TRIGGER H1N1_concern_trigger BEFORE INSERT ON H1N1_Flu_Vaccines
FOR EACH ROW 
BEGIN IF NEW.alert <=3 THEN
SIGNAL SQLSTATE '45000'
SET MESSAGE_TEXT = 'H1N1 concern should be a numerical value between 0 and 3. Please try again.'
;END IF;
END; $$


## Thank you! 